In [ ]:
# WARNING: advised to install a specific version, e.g. ampform==0.1.2
%pip install -q ampform[doc,viz] IPython

In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# Spin alignment

In [ ]:
import logging

import graphviz
import qrules
import sympy as sp
from IPython.display import Math

import ampform
from ampform.io import improve_latex_rendering

LOGGER = logging.getLogger()
LOGGER.setLevel(logging.ERROR)
improve_latex_rendering()

As described in {doc}`compwa-org:report/015`, the {doc}`'standard' helicity formalism </usage/helicity/formalism>` is not suited for state transitions that have different decay topologies. For this reason, the {class}`.HelicityAmplitudeBuilder` can insert a number of Wigner-$D$ function into the amplitude model in case there is more than one underlying {class}`~qrules.topology.Topology`. It is easiest to see this by inspecting the resulting {attr}`.HelicityModel.intensity` and its {attr}`~.HelicityModel.amplitudes`:

In [ ]:
reaction = qrules.generate_transitions(
    initial_state=("J/psi(1S)", [-1, +1]),
    final_state=["K0", "Sigma+", "p~"],
    allowed_intermediate_particles=["Sigma(1660)", "N(1650)"],
    allowed_interaction_types=["strong"],
    formalism="helicity",
)

In [ ]:
src = qrules.io.asdot(
    reaction,
    collapse_graphs=True,
    render_initial_state_id=True,
)
graphviz.Source(src)

'Spin alignment' can be now switched off or on by setting {attr}`.HelicityAmplitudeBuilder.align_spin`:

In [ ]:
builder = ampform.get_builder(reaction)
builder.align_spin = True
model = builder.formulate()
model.intensity

In [ ]:
latex = sp.multiline_latex(
    sp.Symbol("I"),
    model.intensity.evaluate(),
    environment="eqnarray",
)
Math(latex)

This also introduces several **Wigner rotation angles** to the {attr}`.HelicityModel.kinematic_variables`:

In [ ]:
alpha = sp.Symbol("alpha_0^01", real=True)
model.kinematic_variables[alpha]

For more information about these angles, see {ref}`compwa-org:report/015:Compute Wigner rotation angles` in TR-015.

By default, {attr}`~.HelicityAmplitudeBuilder.align_spin` is set to {obj}`False` and the total {attr}`.HelicityModel.intensity` does not contain alignment Wigner-$D$ functions:

In [ ]:
builder.align_spin = False
non_aligned_model = builder.formulate()
non_aligned_model.intensity